# Chemical Reaction Network Theory

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Documents/repos/JuliaTutorials`


In [ ]:
using Catalyst
using LinearAlgebra
import CairoMakie, GraphMakie, NetworkLayout

## Absolute Concentration Robustness (ACR)
Shinar and Feinberg (2010)

In [ ]:
rn = @reaction_network begin
    α, A + B --> 2B
    β, B --> A
end

In [ ]:
plot_complexes(rn)


In [ ]:
reactioncomplexes(rn)


### Linkage classes

In [ ]:
lcs = linkageclasses(rn)


In [ ]:
subnets = subnetworks(rn)
reactions.(subnets)

### Deficiency

In [ ]:
conservationlaws(rn)

conservedequations(rn)

## Independent Decomposition

In [ ]:
# TODO